In [1]:
import os
import uuid
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()


# ✅ Assign the API key
openai_api_key = os.getenv("OPENAI_API_KEY2")

# ✅ Use it here
openai_client = OpenAI(
    api_key=openai_api_key
)

# ✅ Qdrant setup
client = QdrantClient(
    host="vps.maestri.com.co",
    port=6333,
    https=False
)

collection_name = "maestri_knowledge"
embedding_size = 1536  # For OpenAI text-embedding-3-small model

# ✅ Recreate Qdrant collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
)

# ✅ Business documents
documents = [
    {"category": "contacto", "text": "Horarios: Lunes a Viernes de 10:00 A.M a 6:30 P.M. Sábados y domingos de 10:00 A.M a 6:00 P.M."},
    {"category": "contacto", "text": "Dirección: CRA 27A # 68-87 Bogotá DC Colombia. Email: josue.santamaria@maestri.co. Whatsapp: (1) 7328309 - (+57) 3154831517."},
    {"category": "nosotros", "text": "Maestri Milano es una boutique de vinos y alimentos gourmet que lleva a tu mesa una auténtica experiencia gastronómica a la italiana."},
    {"category": "nosotros", "text": "Desde 2019 producimos charcutería artesanal italiana en Colombia con estándares y recetas milenarias italianas."},
    {"category": "nosotros", "text": "Ofrecemos vinos de más de 40 denominaciones y 80 referencias exclusivas, junto a alimentos como Parmigiano Reggiano, trufa y aceite extra virgen."},
    {"category": "descuentos", "text": "Ofrecemos descuentos por cantidad, 5% de descuento en compras de 6 botellas, 10% en compras de 12 botellas"},
    {"category": "servicios", "text": "Ofrecemos servicios de catas de vino y charcutería, asesoría en selección de productos, y catering para eventos. Habla con un asesor para más información."},
    {
        "category": "productos",
        "text": "En Maestri Milano encontrarás una exclusiva selección de productos italianos gourmet: vinos importados y vinos de la casa, charcutería, quesos, pastas, trufas, galletas italianas, carnes curadas, panettone, alimentos gourmet, anchetas, cajas de regalo, cristalería y acceso al Maestri Wine Club. Cada producto ha sido cuidadosamente elegido para brindarte una experiencia auténtica y sofisticada."
    },
    {
        "category": "tiempo de entrega",
        "text": "Ofrecemos dos opciones de envío: Envío normal por $8.000 con entrega entre 24 y 48 horas, o Envío express por $12.000 con entrega rápida entre 3 y 6 horas. Elige la opción que mejor se adapte a tu necesidad."
    },
    {"category": "equipo", "text": "Riccardo Trentini – Sales & Finance"},
    {"category": "equipo", "text": "Carlo Angius – Process Optimization & Markets"},
    {"category": "equipo", "text": "Danilo Marotta – Administración y Producción"},
    {"category": "equipo", "text": "Manuel Ghiddi – Maestro Charcutero"}
]

# ✅ Prepare texts for batch embedding
texts = [doc["text"] for doc in documents]

# ✅ Get embeddings from OpenAI
response = openai_client.embeddings.create(
    input=texts,
    model="text-embedding-3-small"
)

# ✅ Prepare points for Qdrant
points = []
for i, embedding_obj in enumerate(response.data):
    vector = embedding_obj.embedding
    point_id = str(uuid.uuid4())
    payload = {
        "text": documents[i]["text"],
        "category": documents[i]["category"]
    }
    points.append(PointStruct(id=point_id, vector=vector, payload=payload))

# ✅ Upsert into Qdrant
client.upsert(collection_name=collection_name, points=points)

print(f"✅ Inserted {len(points)} documents into Qdrant collection: {collection_name}")


C:\Users\dhernandez\AppData\Local\Temp\ipykernel_31716\529593442.py:30: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Inserted 13 documents into Qdrant collection: maestri_knowledge
